In [ ]:
import cv2
import numpy as np
import math

# Open the webcam
cap0 = cv2.VideoCapture(0) # https://www.logitech.com/en-us/shop/p/c922-pro-stream-webcam.960-001087

# Read one frame to get image size
ret, frame = cap0.read()
cap0.release()

if not ret:
    raise RuntimeError("Could not read from the webcam.")

# Image size
height, width = frame.shape[:2]
print(f"Image resolution: {width} x {height}")

# Estimated horizontal FoV in degrees (you can adjust this if needed)
fov_x_deg = 78
fov_x_rad = math.radians(fov_x_deg)

# Compute focal length in pixels
f_x = width / (2 * math.tan(fov_x_rad / 2))
f_y = f_x  # assume square pixels

# Assume principal point is at the center
c_x = width / 2
c_y = height / 2

# Intrinsic camera matrix
K = np.array([
    [f_x,   0,    c_x],
    [0,     f_y,  c_y],
    [0,     0,    1]
])

print("Approximate Intrinsic Camera Matrix (K):")
print(K)

K0 = K


Image resolution: 640 x 480
Approximate Intrinsic Camera Matrix (K):
[[395.16709009   0.         320.        ]
 [  0.         395.16709009 240.        ]
 [  0.           0.           1.        ]]


In [11]:
import cv2
import numpy as np
import math

# Open the webcam
cap1 = cv2.VideoCapture(1) #laptop cam

# Read one frame to get image size
ret, frame = cap1.read()
cap1.release()

if not ret:
    raise RuntimeError("Could not read from the webcam.")

# Image size
height, width = frame.shape[:2]
print(f"Image resolution: {width} x {height}")

# Estimated horizontal FoV in degrees (you can adjust this if needed)
fov_x_deg = 78.3
fov_x_rad = math.radians(fov_x_deg)

# Compute focal length in pixels
f_x = width / (2 * math.tan(fov_x_rad / 2))
f_y = f_x  # assume square pixels

# Assume principal point is at the center
c_x = width / 2
c_y = height / 2

# Intrinsic camera matrix
K = np.array([
    [f_x,   0,    c_x],
    [0,     f_y,  c_y],
    [0,     0,    1]
])

print("Approximate Intrinsic Camera Matrix (K):")
print(K)

K1 = K


Image resolution: 640 x 480
Approximate Intrinsic Camera Matrix (K):
[[393.05858722   0.         320.        ]
 [  0.         393.05858722 240.        ]
 [  0.           0.           1.        ]]


In [13]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as Rscipy

# Open both cameras
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)

ret0, img0 = cap0.read()
ret1, img1 = cap1.read()

cap0.release()
cap1.release()

if not ret0 or not ret1:
    raise RuntimeError("Could not read from one or both cameras.")

# Convert to grayscale
gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

# Detect and match features
orb = cv2.ORB_create(1000)
kp0, des0 = orb.detectAndCompute(gray0, None)
kp1, des1 = orb.detectAndCompute(gray1, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des0, des1)
matches = sorted(matches, key=lambda x: x.distance)

good_matches = matches[:100]
pts0 = np.float32([kp0[m.queryIdx].pt for m in good_matches])
pts1 = np.float32([kp1[m.trainIdx].pt for m in good_matches])

# Intrinsics
K = np.array([
    [395.16709009, 0, 320],
    [0, 395.16709009, 240],
    [0, 0, 1]
])

# Estimate essential matrix and recover pose
E, mask = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
_, R, t, mask_pose = cv2.recoverPose(E, pts0, pts1, K)

# ---- Part 1: Draw matches ----
matched_img = cv2.drawMatches(img0, kp0, img1, kp1, good_matches, None, flags=2)
cv2.imshow("Feature Matches", matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ---- Part 2: Convert Rotation Matrix to Euler Angles ----
r = Rscipy.from_matrix(R)
euler_angles = r.as_euler('xyz', degrees=True)

print("\nRotation Matrix (R):")
print(R)

print("\nTranslation Vector (t):")
print(t)

print("\nEuler Angles (XYZ, degrees):")
print(euler_angles)



Rotation Matrix (R):
[[ 0.9825454   0.00946636 -0.18578192]
 [ 0.05104048  0.94665734  0.31817409]
 [ 0.17888377 -0.32210289  0.92965065]]

Translation Vector (t):
[[ 0.98306535]
 [-0.07315552]
 [ 0.16802022]]

Euler Angles (XYZ, degrees):
[-19.11004224 -10.30474935   2.97368241]


In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLOv11 model for pose estimation
model = YOLO("yolo11n-pose.pt")

# Open two video captures (e.g., two webcams)
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)

if not cap0.isOpened() or not cap1.isOpened():
    print("Error: Cannot open one or both webcams.")
    exit()

while True:
    # Read frames from both cameras
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    
    if not ret0 or not ret1:
        print("Error: Can't read frames from one or both cameras.")
        break

    # Run pose estimation on both frames
    results0 = model(frame0)
    results1 = model(frame1)

    # Get keypoints from both results (assuming single person detection for simplicity)
    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    # Plot the results (draw skeleton/keypoints)
    annotated_frame0 = results0[0].plot()
    annotated_frame1 = results1[0].plot()

    # Combine frames side by side for display
    combined_frame = np.hstack((annotated_frame0, annotated_frame1))

    # If we have keypoints from both cameras, process them
    if kpts0 is not None and kpts1 is not None:
        # Filter out [0,0] points and create matched pairs
        valid_pairs = []
        for i, (kp0, kp1) in enumerate(zip(kpts0, kpts1)):
            # Skip if either point is [0,0]
            if np.all(kp0 == [0,0]) or np.all(kp1 == [0,0]):
                continue
            valid_pairs.append((i, kp0, kp1))

        # Draw lines between matched keypoints (for visualization)
        for pair in valid_pairs:
            idx, kp0, kp1 = pair
            # Adjust coordinates for the combined frame (kp1 is in the right half)
            pt1 = tuple(map(int, kp0))
            pt2 = (frame0.shape[1] + int(kp1[0]), int(kp1[1]))
            
            # Draw line between matched points
            cv2.line(combined_frame, pt1, pt2, (0, 255, 0), 2)
            
            # Label with the keypoint index
            cv2.putText(combined_frame, str(idx), pt1, cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(combined_frame, str(idx), pt2, cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Display the combined frame
    cv2.imshow("Dual Camera Pose Estimation with Matched Keypoints", combined_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap0.release()
cap1.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 69.4ms
Speed: 3.1ms preprocess, 69.4ms inference, 82.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.4ms
Speed: 2.2ms preprocess, 8.4ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.0ms
Speed: 1.7ms preprocess, 8.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.3ms
Speed: 1.6ms preprocess, 8.3ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.2ms
Speed: 1.4ms preprocess, 8.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.4ms
Speed: 1.1ms preprocess, 9.4ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.9ms
Speed: 1.7ms preprocess, 7.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.1ms
Speed: 0.9ms preprocess, 8.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 4

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from scipy.spatial.transform import Rotation as Rscipy
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---- STEP 1: Estimate Extrinsics (R, t) once ----
def estimate_camera_pose(cap0, cap1, K):
    ret0, img0 = cap0.read()
    ret1, img1 = cap1.read()
    if not ret0 or not ret1:
        raise RuntimeError("Failed to capture calibration images")

    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create(1000)
    kp0, des0 = orb.detectAndCompute(gray0, None)
    kp1, des1 = orb.detectAndCompute(gray1, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des0, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = matches[:100]

    pts0 = np.float32([kp0[m.queryIdx].pt for m in matches])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, pts0, pts1, K)
    return R, t

# ---- STEP 2: Main loop ----
K0 = np.array([[395.16709009, 0, 320],
               [0, 395.16709009, 240],
               [0, 0, 1]])
K1 = np.array([[393.05858722, 0, 320],
               [0, 393.05858722, 240],
               [0, 0, 1]])

# Projection matrices
P0 = K0 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # [I|0]

cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
if not cap0.isOpened() or not cap1.isOpened():
    raise RuntimeError("Cameras could not be opened.")

# Estimate extrinsics once
R, t = estimate_camera_pose(cap0, cap1, K0)
P1 = K1 @ np.hstack((R, t))  # [R|t]

print(R,t)

model = YOLO("yolo11n-pose.pt")

while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        break

    results0 = model(frame0)
    results1 = model(frame1)

    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    annotated_frame0 = results0[0].plot()
    annotated_frame1 = results1[0].plot()
    combined_frame = np.hstack((annotated_frame0, annotated_frame1))

    if kpts0 is not None and kpts1 is not None:
        valid_pairs = []
        for i, (kp0, kp1) in enumerate(zip(kpts0, kpts1)):
            if np.all(kp0 == [0, 0]) or np.all(kp1 == [0, 0]):
                continue
            valid_pairs.append((i, kp0, kp1))

        for pair in valid_pairs:
            idx, kp0, kp1 = pair
            
            # Triangulate 3D point
            kp0_h = np.array([[kp0[0]], [kp0[1]]])
            kp1_h = np.array([[kp1[0]], [kp1[1]]])
            point_4d_h = cv2.triangulatePoints(P0, P1, kp0_h, kp1_h)
            point_3d = point_4d_h[:3] / point_4d_h[3]
            distance = np.linalg.norm(point_3d)

            # Draw connecting lines
            pt1 = tuple(map(int, kp0))
            pt2 = (frame0.shape[1] + int(kp1[0]), int(kp1[1]))
            cv2.line(combined_frame, pt1, pt2, (0, 255, 0), 2)
            cv2.putText(combined_frame, str(idx), pt1, cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(combined_frame, str(idx), pt2, cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 0, 0), 1, cv2.LINE_AA)

            # Draw circles and bigger distance text ONLY for left and right hands (9 and 10)
            if idx == 9 or idx == 10:
                # Scale distance between 1m and 3m:
                # clamp distance between 1 and 3 meters
                dist_clamped = np.clip(distance, 1.0, 3.0)
                
                # Map 1m->max_radius, 3m->min_radius
                max_radius = 40
                min_radius = 10
                radius = int(np.interp(dist_clamped, [1.0, 3.0], [max_radius, min_radius]))
                
                # Choose color and position
                if idx == 9:  # left hand on left frame
                    center = pt1
                    color = (0, 0, 255)  # Red
                else:  # right hand on right frame
                    center = pt2
                    color = (255, 0, 0)  # Blue
                
                cv2.circle(combined_frame, center, radius, color, thickness=2)

                # Bigger font for distance text
                cv2.putText(combined_frame, f"{distance:.2f}m", (center[0], center[1] - radius - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2, cv2.LINE_AA)



    cv2.imshow("Triangulated Pose with Distance", combined_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap0.release()
cap1.release()
cv2.destroyAllWindows()


[[    0.98522   -0.027018    -0.16915]
 [   0.071788      0.9617     0.26452]
 [    0.15552    -0.27276     0.94943]] [[    0.97746]
 [   0.078744]
 [    0.19587]]

0: 480x640 1 person, 9.0ms
Speed: 1.9ms preprocess, 9.0ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.8ms
Speed: 1.0ms preprocess, 9.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.4ms
Speed: 1.4ms preprocess, 8.4ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.9ms
Speed: 1.4ms preprocess, 8.9ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.9ms
Speed: 1.3ms preprocess, 7.9ms inference, 1.8ms postprocess per 